---
## 8. Key Observations & Next Steps

### ✅ What We Achieved:

1. **Concatenation Approach**: Combined LogFile and UsnJrnl without creating duplicate records
2. **No Data Loss**: Every event from both artifacts preserved as separate rows
3. **Separate Label Columns**: Track detection from both artifacts:
   - `is_timestomped_lf` and `is_timestomped_usn`
   - `is_suspicious_lf` and `is_suspicious_usn`
   - Plus unified versions: `is_timestomped`, `is_suspicious`
4. **Source Tracking**: `source_artifact` column identifies whether row came from LogFile or UsnJrnl
5. **Perfect Preservation**: All suspicious records maintained (no duplicates, no losses)

### 📊 Column Structure in Merged Files:

```
timestamp          - Normalized timestamp
filepath           - File path
filename           - File name
source_artifact    - 'logfile' or 'usnjrnl' (identifies source)
lf_lsn             - LogFile LSN (NaN for UsnJrnl rows)
lf_event           - LogFile event type (NaN for UsnJrnl rows)
... (other lf_* columns)
usn_usn            - UsnJrnl USN (NaN for LogFile rows)
usn_event_info     - UsnJrnl event info (NaN for LogFile rows)
... (other usn_* columns)
is_timestomped_lf  - Timestomping in LogFile (NaN for UsnJrnl-only rows)
is_timestomped_usn - Timestomping in UsnJrnl (NaN for LogFile-only rows)
is_suspicious_lf   - Suspicious in LogFile (NaN for UsnJrnl-only rows)
is_suspicious_usn  - Suspicious in UsnJrnl (NaN for LogFile-only rows)
is_timestomped     - Unified timestomping flag
is_suspicious      - Unified suspicious flag
label_source       - Where labels came from
case_id            - Case identifier
```

### 🎯 Why Concatenation Instead of Join?

**Problem with Join Approach:**
- UsnJrnl can have multiple events at same timestamp for same file (e.g., File_Created → Data_Added → File_Closed)
- Join created **duplicates**: 1 LogFile record × 3 UsnJrnl records = 3 rows with same LogFile data
- This inflated suspicious counts incorrectly

**Solution with Concatenation:**
- Each event is a separate row (no duplicates)
- LogFile events have LogFile columns filled, UsnJrnl columns NaN
- UsnJrnl events have UsnJrnl columns filled, LogFile columns NaN
- `source_artifact` column clearly identifies the source

### 📁 Output Files:

All merged files saved to:
`data/processed/Phase 1 - Data Collection & Preprocessing/B. Data Case Merging/XX-PE-Merged.csv`

### ➡️ Next Steps:

**Phase 1C: Master Timeline Creation**
- Concatenate all 12 merged case files into single master timeline
- Sort by timestamp for chronological analysis
- Prepare for feature engineering (Phase 2)

In [ ]:
# Load merged 01-PE to verify
sample_merged = pd.read_csv(OUTPUT_DIR / "01-PE-Merged.csv", encoding='utf-8-sig')

# Find NEWFILETIME_X64.EXE-6C60D39A.pf records
newfiletime_records = sample_merged[sample_merged['filename'] == 'NEWFILETIME_X64.EXE-6C60D39A.pf']

print("=" * 80)
print("VERIFICATION: NEWFILETIME_X64.EXE-6C60D39A.pf Records")
print("=" * 80)
print(f"\\nTotal records for this file: {len(newfiletime_records)}")
print(f"\\nMerge type breakdown:")
print(newfiletime_records['merge_type'].value_counts())

# Show matched record (if exists)
matched = newfiletime_records[newfiletime_records['merge_type'] == 'matched']
if len(matched) > 0:
    print(f"\\n✅ SUCCESS! Found {len(matched)} MATCHED record(s) with data from BOTH LogFile and UsnJrnl:")
    display(matched[['timestamp', 'filename', 'lf_lsn', 'usn_usn', 'lf_event', 'usn_event_info', 
                     'merge_type', 'is_timestomped_lf', 'is_timestomped_usn', 'is_suspicious']])
else:
    print("\\n⚠️ No matched records found")

print(f"\\n📋 All records for this file:")
display(newfiletime_records[['timestamp', 'filename', 'lf_lsn', 'usn_usn', 'lf_event', 'usn_event_info', 'merge_type']])

---
## 7. Verify Example: NEWFILETIME_X64.EXE-6C60D39A.pf

Let's verify that the problematic file mentioned at the start is now properly merged!

In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

x = np.arange(len(stats_df))
width = 0.35

# Plot 1: Before vs After comparison
before = stats_df['lf_records_before'] + stats_df['usn_records_before']
after = stats_df['total_records_after']

axes[0, 0].bar(x - width/2, before, width, label='Before Merge (LF+USN)', color='lightcoral', alpha=0.7)
axes[0, 0].bar(x + width/2, after, width, label='After Merge', color='lightgreen', alpha=0.7)
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(stats_df['case_id'], rotation=45)
axes[0, 0].set_xlabel('Case ID', fontsize=11)
axes[0, 0].set_ylabel('Record Count', fontsize=11)
axes[0, 0].set_title('Dataset Size Reduction per Case', fontsize=13, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Plot 2: Merge type breakdown (stacked)
axes[0, 1].bar(x, stats_df['matched_events'], label='Matched (Both)', color='#4CAF50', alpha=0.7)
axes[0, 1].bar(x, stats_df['logfile_only'], bottom=stats_df['matched_events'], 
               label='LogFile Only', color='#FF9800', alpha=0.7)
axes[0, 1].bar(x, stats_df['usnjrnl_only'], 
               bottom=stats_df['matched_events'] + stats_df['logfile_only'],
               label='UsnJrnl Only', color='#2196F3', alpha=0.7)
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(stats_df['case_id'], rotation=45)
axes[0, 1].set_xlabel('Case ID', fontsize=11)
axes[0, 1].set_ylabel('Record Count', fontsize=11)
axes[0, 1].set_title('Merge Type Distribution per Case', fontsize=13, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Plot 3: Suspicious record preservation
axes[1, 0].bar(x - width/2, stats_df['total_suspicious_before'], width, 
               label='Before Merge', color='#ff6b6b', alpha=0.7)
axes[1, 0].bar(x + width/2, stats_df['suspicious_after'], width, 
               label='After Merge', color='#51cf66', alpha=0.7)
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(stats_df['case_id'], rotation=45)
axes[1, 0].set_xlabel('Case ID', fontsize=11)
axes[1, 0].set_ylabel('Suspicious Count', fontsize=11)
axes[1, 0].set_title('⚠️ Suspicious Record Preservation', fontsize=13, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# Plot 4: Suspicious event coverage breakdown (pie chart)
coverage_data = [total_susp_matched, total_susp_lf_only, total_susp_usn_only]
labels = ['Matched\\n(Full Row)', 'LogFile Only', 'UsnJrnl Only']
colors = ['#4CAF50', '#FF9800', '#2196F3']
explode = (0.1, 0, 0)

axes[1, 1].pie(coverage_data, labels=labels, autopct='%1.1f%%', startangle=90,
               colors=colors, explode=explode, shadow=True)
axes[1, 1].set_title('Suspicious Event Coverage Distribution', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\\n📊 Visualizations complete")

---
## 6. Visualizations

In [ ]:
# Create summary DataFrame
stats_df = pd.DataFrame(all_merge_stats)

print("\n" + "=" * 80)
print("MERGE SUMMARY - ALL CASES (Concatenation)")
print("=" * 80)
print("\n📊 Per-Case Statistics:")
display(stats_df)

print("\n" + "=" * 80)
print("OVERALL STATISTICS")
print("=" * 80)

# Calculate totals
total_lf_before = stats_df['lf_records_before'].sum()
total_usn_before = stats_df['usn_records_before'].sum()
total_after = stats_df['total_records_after'].sum()

print(f"\n📈 Record Statistics:")
print(f"   LogFile records:       {total_lf_before:,}")
print(f"   UsnJrnl records:       {total_usn_before:,}")
print(f"   Total after concat:    {total_after:,}")
print(f"   (Should equal sum):    {total_lf_before + total_usn_before:,} {'✅' if total_after == total_lf_before + total_usn_before else '❌'}")

# Suspicious record preservation
total_suspicious_before = stats_df['total_suspicious_before'].sum()
total_suspicious_after = stats_df['suspicious_after'].sum()
cases_preserved = stats_df['suspicious_preserved'].sum()

print(f"\n⚠️ SUSPICIOUS RECORD PRESERVATION:")
print(f"   Total suspicious before: {total_suspicious_before}")
print(f"   Total suspicious after:  {total_suspicious_after}")
print(f"   Match status:            {'✅ PERFECT MATCH' if total_suspicious_after == total_suspicious_before else '❌ MISMATCH'}")
print(f"   Cases preserved exactly: {cases_preserved}/{len(stats_df)}")

if total_suspicious_after != total_suspicious_before:
    print(f"   ⚠️ WARNING: Expected {total_suspicious_before} but got {total_suspicious_after}!")

print(f"\n💡 KEY INSIGHTS:")
print(f"   1. Concatenation approach: Each event kept as separate row")
print(f"   2. No duplicate LogFile records from one-to-many joins")
print(f"   3. All {total_suspicious_before} suspicious records preserved")
print(f"   4. Dataset has both LogFile and UsnJrnl events (identified by 'source_artifact' column)")

---
## 5. Merge Summary & Analytics

In [ ]:
# Process all cases
all_merge_stats = []

print("\n" + "=" * 80)
print("PROCESSING ALL CASES (Concatenation Approach)")
print("=" * 80)

for case_id in CASE_IDS:
    print(f"\n[{case_id}] Processing...")
    
    try:
        # Merge case
        merged_df, stats = merge_case(case_id, LABELLED_DIR)
        
        print(f"  Records: {stats['lf_records_before']:,} + {stats['usn_records_before']:,} = {stats['total_records_after']:,}")
        print(f"  Suspicious: {stats['total_suspicious_before']} → {stats['suspicious_after']} {'✅' if stats['suspicious_preserved'] else '❌'}")
        
        # Save merged dataset
        output_path = OUTPUT_DIR / f"{case_id}-Merged.csv"
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"  ✓ Saved: {output_path.name}")
        
        # Store statistics
        all_merge_stats.append(stats)
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "=" * 80)
print("✓ ALL CASES PROCESSED")
print("=" * 80)

---
## 4. Process All Cases

In [ ]:
# Test merge on case 01-PE
print("Testing merge on case 01-PE...\n")

test_merged, test_stats = merge_case('01-PE', LABELLED_DIR)

print("=" * 80)
print("MERGE TEST RESULTS: 01-PE (Concatenation Approach)")
print("=" * 80)

print(f"\n📊 Record Counts:")
print(f"   LogFile before:        {test_stats['lf_records_before']:,}")
print(f"   UsnJrnl before:        {test_stats['usn_records_before']:,}")
print(f"   Total after merge:     {test_stats['total_records_after']:,}")
print(f"   From LogFile:          {test_stats['from_logfile']:,}")
print(f"   From UsnJrnl:          {test_stats['from_usnjrnl']:,}")

print(f"\n⚠️ SUSPICIOUS RECORD PRESERVATION:")
print(f"   LogFile suspicious before:  {test_stats['lf_suspicious_before']}")
print(f"   UsnJrnl suspicious before:  {test_stats['usn_suspicious_before']}")
print(f"   Total suspicious before:    {test_stats['total_suspicious_before']}")
print(f"   Suspicious after merge:     {test_stats['suspicious_after']}")
print(f"   Status: {'✅ PRESERVED (EXACT MATCH)' if test_stats['suspicious_preserved'] else '❌ MISMATCH!'}")

if not test_stats['suspicious_preserved']:
    print(f"   ⚠️ WARNING: Expected {test_stats['total_suspicious_before']} but got {test_stats['suspicious_after']}!")

print(f"\n📋 Sample of merged data:")
display(test_merged[['timestamp', 'filepath', 'filename', 'source_artifact', 
                      'lf_lsn', 'usn_usn', 'lf_event', 'usn_event_info',
                      'is_timestomped_lf', 'is_timestomped_usn', 'is_suspicious']].head(15))

---
## 3. Test Merge on Sample Case (01-PE)

In [ ]:
def merge_case(case_id, labelled_dir):
    """
    Merge LogFile and UsnJrnl for a single case using CONCATENATION approach.
    This prevents duplicate records from one-to-many relationships.
    Returns: (merged_df, merge_stats_dict)
    """
    # Load labelled datasets
    logfile_path = labelled_dir / f"{case_id}-LogFile-Labelled.csv"
    usnjrnl_path = labelled_dir / f"{case_id}-UsnJrnl-Labelled.csv"
    
    logfile_df = pd.read_csv(logfile_path, encoding='utf-8-sig')
    usnjrnl_df = pd.read_csv(usnjrnl_path, encoding='utf-8-sig')
    
    # Store original counts
    lf_suspicious_before = logfile_df['is_suspicious'].sum()
    usn_suspicious_before = usnjrnl_df['is_suspicious'].sum()
    total_suspicious_before = lf_suspicious_before + usn_suspicious_before
    
    # Prepare for merge with normalized timestamps
    lf_prepared = prepare_logfile_for_merge(logfile_df)
    usn_prepared = prepare_usnjrnl_for_merge(usnjrnl_df)
    
    # Add source identifier
    lf_prepared['source_artifact'] = 'logfile'
    usn_prepared['source_artifact'] = 'usnjrnl'
    
    # Concatenate instead of merge to avoid duplicates
    # This keeps each event as a separate row
    merged = pd.concat([lf_prepared, usn_prepared], ignore_index=True)
    
    # Sort by timestamp and filepath for better organization
    merged = merged.sort_values(['timestamp', 'filepath'], na_position='last').reset_index(drop=True)
    
    # Create unified label columns (already exist from preparation)
    # For concatenation, just rename back to standard names
    merged['is_timestomped'] = merged['is_timestomped_lf'].fillna(merged['is_timestomped_usn'])
    merged['is_suspicious_execution'] = merged['is_suspicious_execution_lf'].fillna(merged['is_suspicious_execution_usn'])
    merged['is_suspicious'] = merged['is_suspicious_lf'].fillna(merged['is_suspicious_usn'])
    merged['label_source'] = merged['label_source_lf'].fillna(merged['label_source_usn'])
    
    # Add case_id
    merged['case_id'] = case_id
    
    # Gather statistics
    suspicious_after = merged['is_suspicious'].sum()
    
    merge_stats = {
        'case_id': case_id,
        'lf_records_before': len(logfile_df),
        'usn_records_before': len(usnjrnl_df),
        'total_records_after': len(merged),
        'lf_suspicious_before': int(lf_suspicious_before),
        'usn_suspicious_before': int(usn_suspicious_before),
        'total_suspicious_before': int(total_suspicious_before),
        'suspicious_after': int(suspicious_after),
        'suspicious_preserved': suspicious_after == total_suspicious_before,
        'from_logfile': len(merged[merged['source_artifact'] == 'logfile']),
        'from_usnjrnl': len(merged[merged['source_artifact'] == 'usnjrnl'])
    }
    
    return merged, merge_stats


print("✓ Merge function defined (concatenation approach)")

In [ ]:
def normalize_timestamp(ts):
    """
    Normalize timestamp to consistent format with leading zeros.
    Fixes the issue where '12/23/23 0:21:57' != '12/23/23 00:21:57'
    """
    if pd.isna(ts) or ts == '':
        return ts
    
    try:
        # Parse and reformat with leading zeros
        dt = pd.to_datetime(ts, format='%m/%d/%y %H:%M:%S', errors='coerce')
        if pd.isna(dt):
            dt = pd.to_datetime(ts, errors='coerce')
        if pd.notna(dt):
            return dt.strftime('%m/%d/%y %H:%M:%S')
        return ts
    except:
        return ts


def prepare_logfile_for_merge(df):
    """
    Prepare LogFile for merging: 
    - Normalize timestamps
    - Add 'lf_' prefix to columns
    - Keep separate label columns
    """
    prepared = df.copy()
    
    # Standardize join keys with normalized timestamp
    prepared['timestamp'] = prepared['EventTime(UTC+8)'].apply(normalize_timestamp)
    prepared['filepath'] = prepared['Full Path']
    prepared['filename'] = prepared['File/Directory Name']
    
    # Rename with 'lf_' prefix
    rename_map = {
        'LSN': 'lf_lsn',
        'Event': 'lf_event',
        'Detail': 'lf_detail',
        'CreationTime': 'lf_creation_time',
        'ModifiedTime': 'lf_modified_time',
        'MFTModifiedTime': 'lf_mft_modified_time',
        'AccessedTime': 'lf_accessed_time',
        'Redo': 'lf_redo',
        'Target VCN': 'lf_target_vcn',
        'Cluster Index': 'lf_cluster_index',
        # Keep label columns separate with lf_ prefix
        'is_timestomped': 'is_timestomped_lf',
        'is_suspicious_execution': 'is_suspicious_execution_lf',
        'is_suspicious': 'is_suspicious_lf',
        'label_source': 'label_source_lf'
    }
    prepared = prepared.rename(columns=rename_map)
    prepared = prepared.drop(['EventTime(UTC+8)', 'Full Path', 'File/Directory Name'], axis=1, errors='ignore')
    
    return prepared


def prepare_usnjrnl_for_merge(df):
    """
    Prepare UsnJrnl for merging:
    - Normalize timestamps
    - Add 'usn_' prefix to columns
    - Keep separate label columns
    """
    prepared = df.copy()
    
    # Standardize join keys with normalized timestamp
    prepared['timestamp'] = prepared['TimeStamp(UTC+8)'].apply(normalize_timestamp)
    prepared['filepath'] = prepared['FullPath']
    prepared['filename'] = prepared['File/Directory Name']
    
    # Rename with 'usn_' prefix
    rename_map = {
        'USN': 'usn_usn',
        'EventInfo': 'usn_event_info',
        'SourceInfo': 'usn_source_info',
        'FileAttribute': 'usn_file_attribute',
        'Carving Flag': 'usn_carving_flag',
        'FileReferenceNumber': 'usn_file_reference_number',
        'ParentFileReferenceNumber': 'usn_parent_file_reference_number',
        # Keep label columns separate with usn_ prefix
        'is_timestomped': 'is_timestomped_usn',
        'is_suspicious_execution': 'is_suspicious_execution_usn',
        'is_suspicious': 'is_suspicious_usn',
        'label_source': 'label_source_usn'
    }
    prepared = prepared.rename(columns=rename_map)
    prepared = prepared.drop(['TimeStamp(UTC+8)', 'FullPath', 'File/Directory Name'], axis=1, errors='ignore')
    
    return prepared


print("✓ Preparation functions defined")

---
## 2. Merge Functions with Timestamp Normalization

In [ ]:
# Case IDs
CASE_IDS = [f"{i:02d}-PE" for i in range(1, 13)]
print(f"Processing {len(CASE_IDS)} cases: {', '.join(CASE_IDS)}")

In [ ]:
# Define paths
notebook_dir = Path.cwd()
print(f"Current working directory: {notebook_dir}")

# Navigate to project root
if 'notebooks' in str(notebook_dir):
    BASE_DIR = notebook_dir.parent.parent / 'data'
else:
    BASE_DIR = Path('data')

LABELLED_DIR = BASE_DIR / 'processed' / 'Phase 1 - Data Collection & Preprocessing' / 'A. Data Labelled'
OUTPUT_DIR = BASE_DIR / 'processed' / 'Phase 1 - Data Collection & Preprocessing' / 'B. Data Case Merging'

# Ensure output directory exists
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n📂 Directory Configuration:")
print(f"  Input:  {LABELLED_DIR} {'✓' if LABELLED_DIR.exists() else '✗ NOT FOUND'}")
print(f"  Output: {OUTPUT_DIR} ✓")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Libraries imported successfully")

---
## 1. Setup & Imports

# Phase 1B: Data Case Merging

## Objective
Merge LogFile and UsnJrnl labelled artifacts per case by **intelligently joining** rows that represent the same forensic event.

## Problem Solved
Previously, the same file system event recorded in **both** LogFile and UsnJrnl appeared as **two separate rows**:
- LogFile row: `12/23/23 0:21:57 | \Windows\Prefetch\NEWFILETIME_X64.EXE-6C60D39A.pf`
- UsnJrnl row: `12/23/23 00:21:57 | \Windows\Prefetch\NEWFILETIME_X64.EXE-6C60D39A.pf`

**Issue**: Timestamps formatted differently (`0:21:57` vs `00:21:57`) prevented matching!

## Solution: Intelligent Join with Timestamp Normalization
✅ Normalize timestamps to consistent format  
✅ Merge rows with same timestamp + filepath  
✅ Keep **separate** label columns for each artifact (`is_timestomped_lf`, `is_timestomped_usn`)  
✅ Track where suspicious activity was detected  

## Expected Results
- **Significant dataset reduction**: ~4M → ~350K rows
- **Better data quality**: Matched events have complete LogFile + UsnJrnl data
- **Preserved detection information**: Know which artifact detected each suspicious activity

## Example Merged Row
```
timestamp         | lf_lsn     | usn_usn    | filename                         | filepath                                     | lf_event      | usn_event_info                    | is_timestomped_lf | is_timestomped_usn | ...
12/23/23 00:21:57 | 8729569062 | 1327928416 | NEWFILETIME_X64.EXE-6C60D39A.pf | \Windows\Prefetch\NEWFILETIME_X64.EXE-...pf | File Creation | File_Created / Data_Added / ...   | 0                 | 1                  | ...
```